In [3]:
# -*- coding: utf-8 -*-
"""
report_all.csv를 '그대로' 읽어 종합 비교 시각화를 생성합니다.
- 파일 변환/저장(정규화 CSV 등) 없이, 메모리 내 가공만 수행하고 그림(PNG)만 저장합니다.
- 개선점:
  1) 'original'이 표에 안 보이던 문제를 해결 (feature 표준화 + 결측시 original로 대체)
  2) 세트 F는 f1/AUPRC/AUROC/Brier/ECE 5개 지표를 한 장의 멀티 패널로 한 번에 시각화
- VS Code / Jupyter에서 그대로 실행하세요.
"""

import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------- 설정 --------------------
CSV_PATH = "report_all.csv"          # 현재 폴더에 CSV가 있다고 가정
OUT_DIR  = Path("figs_compare")      # 출력 이미지 저장 폴더
OUT_DIR.mkdir(parents=True, exist_ok=True)

SETS = ["O", "F", "OF"]
METRIC_ORDER = ["f1", "AUPRC", "AUROC", "Brier", "ECE"]
FEATURE_ORDER = ["original", "m", "mw", "m+mw"]

# 알고리즘(algo)이 여러 개일 때 집계 방식 (평균)
AGGREGATE_OVER_ALGO = True

# 지표 최적화 방향
HIGHER_IS_BETTER = {"f1": True, "AUPRC": True, "AUROC": True, "Brier": False, "ECE": False}

# (옵션) 한글 폰트 설정 - 필요 시 주석 처리
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

# -------------------- 유틸 --------------------
def try_read_csv(path: str) -> pd.DataFrame:
    for enc in ["utf-8-sig", "utf-8", "cp949", "euc-kr"]:
        try:
            return pd.read_csv(path, encoding=enc, dtype=str)
        except Exception:
            pass
    return pd.read_csv(path, dtype=str)

def norm_metric_name(x: str) -> str:
    if not isinstance(x, str):
        return ""
    t = x.strip().lower()
    if t in ["f1", "f1-score", "f1score"]: return "f1"
    if t in ["auprc", "apr", "average precision"]: return "AUPRC"
    if t in ["auroc", "rocauc", "roc-auc", "auc"]: return "AUROC"
    if t in ["brier", "brier score", "brier_score"]: return "Brier"
    if t in ["ece", "expected calibration error", "calibration"]: return "ECE"
    return x.strip()

def norm_feature_name(x) -> str:
    """
    feature 값을 표준화:
    - 공백/대소문자/변형(synonym) 정리
    - 결측/빈 문자열/헤더 텍스트는 'original'로 대체
    """
    if x is None:
        return "original"
    s = str(x).strip()
    if s == "" or s.lower() in ["nan", "none", "null", "feature"]:
        return "original"
    t = s.lower().replace(" ", "")
    t = t.replace("m+w", "m+mw").replace("m_plus_mw", "m+mw").replace("mplusmw", "m+mw")
    # 흔한 variant 매핑
    table = {
        "orig": "original", "ori": "original", "baseline": "original",
        "original": "original", "org": "original",
        "m": "m", "mw": "mw", "m+mw": "m+mw", "m&m": "m+mw"
    }
    return table.get(t, s)  # 모르는 값은 원문 유지

def safe_filename(name: str) -> str:
    return "".join(ch if ch.isalnum() or ch in ("-", "_", ".") else "_" for ch in str(name))

# -------------------- 1) CSV 로드 --------------------
raw = try_read_csv(CSV_PATH)

# id 컬럼 추정: 'Unnamed: 0/1/2' → algo / model / feature
rename_map = {}
for c in raw.columns:
    if c.startswith("Unnamed: 0"):
        rename_map[c] = "algo"
    elif c.startswith("Unnamed: 1"):
        rename_map[c] = "model"
    elif c.startswith("Unnamed: 2"):
        rename_map[c] = "feature"
df = raw.rename(columns=rename_map).copy()

# 헤더 행(예: model == 'Model' 또는 feature == 'Feature') 탐지 → 해당 행은 데이터에서 제외
header_like = pd.Series(False, index=df.index)
if "model" in df.columns:
    header_like |= df["model"].astype(str).str.fullmatch(r"Model", na=False)
if "feature" in df.columns:
    header_like |= df["feature"].astype(str).str.fullmatch(r"Feature", na=False)

hdr_idx = header_like[header_like].index[0] if header_like.any() else 0
hdr_row = df.iloc[hdr_idx] if len(df) > 0 else pd.Series(dtype=str)

# feature 표준화 (← original 누락 이슈 해결의 핵심)
if "feature" in df.columns:
    df["feature"] = df["feature"].map(norm_feature_name)
else:
    df["feature"] = "original"  # 없으면 모두 original

# model/algo 정리(앞뒤 공백 제거)
if "model" in df.columns:
    df["model"] = df["model"].astype(str).str.strip()
if "algo" in df.columns:
    df["algo"] = df["algo"].astype(str).str.strip()

# -------------------- 2) 롱 포맷 구성(메모리 내) --------------------
records = []
for set_name in SETS:
    # set에 속한 열 수집
    set_cols = [c for c in df.columns if c == set_name or c.startswith(set_name + ".")]

    # set별 메트릭명 추출(헤더 행 기반) + 위치 기반 보정
    metric_names = []
    for j, col in enumerate(set_cols):
        m = norm_metric_name(str(hdr_row.get(col, "")).strip())
        if m == "" or m.lower() in ["nan", "none"]:
            fallback = {0: "f1", 1: "AUPRC", 2: "AUROC", 3: "Brier", 4: "ECE"}
            m = fallback.get(j, f"metric{j+1}")
        metric_names.append(m)

    # 데이터 행 반복
    for i, row in df.iterrows():
        if i == hdr_idx:  # 헤더 유사행 skip
            continue
        algo = row.get("algo", None)
        model = row.get("model", None)
        feat  = row.get("feature", None)

        for j, col in enumerate(set_cols):
            val_raw = row.get(col, None)
            try:
                val = float(str(val_raw).replace(",", "").strip())
            except Exception:
                continue
            records.append({
                "set": set_name,
                "metric": metric_names[j],
                "algo": algo,
                "model": model,
                "feature": feat,
                "value": val
            })

long_df = pd.DataFrame.from_records(records)
# 필수 키가 비어있는 행 제거
long_df = long_df.dropna(subset=["set", "metric", "model", "feature", "value"])
# feature 정렬 보장
long_df["feature"] = pd.Categorical(long_df["feature"], categories=FEATURE_ORDER, ordered=True)

# -------------------- 3) 시각화 함수 --------------------
def grouped_bar_by_model(df_sub: pd.DataFrame, title: str, out_path: Path):
    """
    X축: model, 막대: feature, 값: value (algo가 여러 개면 평균)
    """
    data = df_sub.copy()
    if AGGREGATE_OVER_ALGO:
        data = data.groupby(["model", "feature"], as_index=False)["value"].mean()

    # 피벗 (모든 feature 컬럼 강제 포함)
    pivot = data.pivot_table(index="model", columns="feature", values="value", aggfunc="mean")
    pivot = pivot.reindex(columns=FEATURE_ORDER)  # 존재하지 않는 컬럼도 순서 유지(전부 NaN이면 그릴 때 생략됨)

    # 그리기
    fig = plt.figure(figsize=(12, 6))
    ax = pivot.plot(kind="bar", figsize=(12, 6), ax=plt.gca())
    ax.set_title(title)
    ax.set_xlabel("Model")
    ax.set_ylabel("Value")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    fig.savefig(out_path, dpi=150, bbox_inches="tight")
    plt.close(fig)

def multi_metric_panel(df_all: pd.DataFrame, set_name: str, metrics=METRIC_ORDER, fname=None):
    """
    한 세트 내 5개 지표를 '한 장'에 그리기 (요청사항)
    - 각 패널: X축=model, 막대=feature
    - 값: (algo 평균) value
    """
    sub_all = df_all[df_all["set"] == set_name]
    if sub_all.empty:
        return
    n = len(metrics)
    fig, axes = plt.subplots(1, n, figsize=(5.2*n, 5), sharey=False)
    if n == 1:
        axes = [axes]

    last_handles, last_labels = None, None

    for ax, m in zip(axes, metrics):
        sub = sub_all[sub_all["metric"] == m]
        if sub.empty:
            ax.axis("off")
            ax.set_title(f"{m} (no data)")
            continue

        data = sub.copy()
        if AGGREGATE_OVER_ALGO:
            data = data.groupby(["model", "feature"], as_index=False)["value"].mean()

        pivot = data.pivot_table(index="model", columns="feature", values="value", aggfunc="mean")
        pivot = pivot.reindex(columns=FEATURE_ORDER)

        pivot.plot(kind="bar", ax=ax)
        ax.set_title(m)
        ax.set_xlabel("")
        ax.set_ylabel("Value")
        ax.tick_params(axis="x", rotation=75)

        last_handles, last_labels = ax.get_legend_handles_labels()

    # 공통 제목 + 범례(아래쪽 중앙)
    fig.suptitle(f"{set_name} — Model vs Feature across Metrics", y=1.02, fontsize=12)
    if last_handles and last_labels:
        fig.legend(last_handles, last_labels, loc="lower center", ncol=min(4, len(last_labels)), bbox_to_anchor=(0.5, -0.01))
    fig.tight_layout()
    out_path = OUT_DIR / (fname if fname else f"{set_name}_all_metrics.png")
    fig.savefig(out_path, dpi=160, bbox_inches="tight")
    plt.close(fig)

# -------------------- 4) 그림 생성 --------------------
# (A) 세트 F: 5개 지표 한 장에
multi_metric_panel(long_df, "F", metrics=METRIC_ORDER, fname="F_all_metrics.png")

# (B) 참고: 단일 지표별 비교 그림도 같이 생성 (세트 O/F/OF 전부)
for set_name in SETS:
    for metric in METRIC_ORDER:
        sub = long_df[(long_df["set"] == set_name) & (long_df["metric"] == metric)]
        if sub.empty:
            continue
        title = f"{set_name} — {metric}: Model vs Feature"
        out_path = OUT_DIR / f"grouped_{set_name}_{safe_filename(metric)}.png"
        grouped_bar_by_model(sub, title, out_path)

# -------------------- 5) 가장 우수한 '모델' 요약(콘솔 출력) --------------------
def best_models_report(df_all: pd.DataFrame):
    """
    각 set × metric 별로 '모델' 종합 점수 산출:
    - 방법 A: (algo 평균) 후 feature 평균 → 모델 단일 점수
    - 방법 B: (algo 평균) 후 feature 중 최상값 → 모델 단일 점수
    """
    lines = []
    for set_name in SETS:
        for metric in METRIC_ORDER:
            g = df_all[(df_all["set"] == set_name) & (df_all["metric"] == metric)]
            if g.empty:
                continue

            # algo 평균
            if AGGREGATE_OVER_ALGO:
                g = g.groupby(["model", "feature"], as_index=False)["value"].mean()

            # 방법 A: feature 평균
            avg_by_model = g.groupby("model", as_index=False)["value"].mean()

            # 방법 B: feature 중 최상값
            better_high = HIGHER_IS_BETTER.get(metric, True)
            best_by_model = (g.groupby("model", as_index=False)["value"].max()
                             if better_high else
                             g.groupby("model", as_index=False)["value"].min())

            avg_sorted = avg_by_model.sort_values("value", ascending=not better_high)
            best_sorted = best_by_model.sort_values("value", ascending=not better_high)

            topA = avg_sorted.iloc[0] if not avg_sorted.empty else None
            topB = best_sorted.iloc[0] if not best_sorted.empty else None

            lines.append(f"[{set_name} / {metric}]")
            if topA is not None:
                lines.append(f" - 방법A(특징 평균) Top: {topA['model']} = {topA['value']:.4f}")
            if topB is not None:
                lines.append(f" - 방법B(최고 특징) Top: {topB['model']} = {topB['value']:.4f}")
            lines.append("")
    print("\n===== 가장 우수한 모델 요약 =====")
    print("\n".join(lines))

best_models_report(long_df)

print(f"\n[완료] 그림이 '{OUT_DIR.resolve()}' 폴더에 저장되었습니다.")
print(" - 핵심: F_all_metrics.png (F 세트 5개 지표 한 장)")
print(" - 참고: grouped_O_*.png / grouped_F_*.png / grouped_OF_*.png")


C:\Users\User\AppData\Local\Temp\ipykernel_25784\1980650608.py:209: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data = data.groupby(["model", "feature"], as_index=False)["value"].mean()
C:\Users\User\AppData\Local\Temp\ipykernel_25784\1980650608.py:211: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = data.pivot_table(index="model", columns="feature", values="value", aggfunc="mean")
C:\Users\User\AppData\Local\Temp\ipykernel_25784\1980650608.py:209: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True t


===== 가장 우수한 모델 요약 =====
[O / f1]
 - 방법A(특징 평균) Top: XGBoost = 0.4039
 - 방법B(최고 특징) Top: XGBoost = 0.4136

[O / AUPRC]
 - 방법A(특징 평균) Top: RF = 0.4471
 - 방법B(최고 특징) Top: RF = 0.4481

[O / AUROC]
 - 방법A(특징 평균) Top: FF MLP = 0.9154
 - 방법B(최고 특징) Top: FF MLP = 0.9155

[O / Brier]
 - 방법A(특징 평균) Top: RF = 0.0160
 - 방법B(최고 특징) Top: RF = 0.0160

[O / ECE]
 - 방법A(특징 평균) Top: RF = 0.0054
 - 방법B(최고 특징) Top: RF = 0.0054

[F / f1]
 - 방법A(특징 평균) Top: XGBoost = 0.3467
 - 방법B(최고 특징) Top: XGBoost = 0.4083

[F / AUPRC]
 - 방법A(특징 평균) Top: XGBoost = 0.3900
 - 방법B(최고 특징) Top: XGBoost = 0.4400

[F / AUROC]
 - 방법A(특징 평균) Top: XGBoost = 0.8619
 - 방법B(최고 특징) Top: XGBoost = 0.9075

[F / Brier]
 - 방법A(특징 평균) Top: LightGBM = 0.0304
 - 방법B(최고 특징) Top: RF = 0.0163

[F / ECE]
 - 방법A(특징 평균) Top: LightGBM = 0.0505
 - 방법B(최고 특징) Top: RF = 0.0053

[OF / f1]
 - 방법A(특징 평균) Top: XGBoost = 0.4184
 - 방법B(최고 특징) Top: XGBoost = 0.4284

[OF / AUPRC]
 - 방법A(특징 평균) Top: XGBoost = 0.4676
 - 방법B(최고 특징) Top: XGBoost = 0.4726

[OF /

C:\Users\User\AppData\Local\Temp\ipykernel_25784\1980650608.py:261: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = g.groupby(["model", "feature"], as_index=False)["value"].mean()
C:\Users\User\AppData\Local\Temp\ipykernel_25784\1980650608.py:261: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = g.groupby(["model", "feature"], as_index=False)["value"].mean()
C:\Users\User\AppData\Local\Temp\ipykernel_25784\1980650608.py:261: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to